In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import types as T
from pyspark.sql import functions as F

from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Create spark session

In [ ]:
spark = SparkSession \
            .builder \
            .master("local[*]")\
            .appName("Loan approval classification using Decision Tree") \
            .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
            .getOrCreate()

25/02/12 00:17:48 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:76)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:53)
java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:500)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:484)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.command

KeyboardInterrupt: 

25/02/12 00:18:09 WARN StandaloneAppClient$ClientEndpoint: Failed to connect to master spark-master:7077
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anon$1.run(StandaloneAppClient.scala:108)
	at java.base/java.util.concurrent.Executors$RunnableAdapter.call(Executors.java:577)
	at java.base/java.util.concurrent.FutureTask.run(FutureTask.java:317)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1144)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:642)
	at j

# Load data from feature store

In [ ]:
df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://172.18.0.1:5444/feast") \
    .option("dbtable", "devices") \
    .option("user", "k6") \
    .option("password", "k6") \
    .option("driver", "org.postgresql.Driver") \
    .load()


# Build, Hyperparameters selection and train model

In [5]:
hyper_parameters = {
    "C": [1, 10],
    "kernel": ["rbf", "linear"],
    "gamma": ["scale", "auto"]
}

In [6]:
# Create a LinearSVC model
model = SVC()

grid_search = GridSearchCV(estimator=model, 
                           param_grid=hyper_parameters, 
                           cv=3, # Number of fold 
                           n_jobs=5,
                           verbose=True)

grid_search.fit(X_train, y_train)

best_model = grid_search.best_estimator_

Fitting 3 folds for each of 8 candidates, totalling 24 fits


# Save model

In [11]:
# Define the initial type of the model input
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

# Convert the model to ONNX
onnx_model = convert_sklearn(best_model, initial_types=initial_type)

# Save the ONNX model to a file
onnx.save_model(onnx_model, str(MODEL_DIR / 'training_experiment_1.onnx'))

# Load model and make prediction 

In [12]:
# Initialize the ONNX Runtime session
session = ort.InferenceSession(str(MODEL_DIR / 'training_experiment_1.onnx'))

# Prepare the input data for prediction
input_name = session.get_inputs()[0].name
X_test_onnx = np.array(X_test, dtype=np.float32)

# Perform inference
y_pred = session.run(None, {input_name: X_test_onnx})[0]

# Evaluation

In [13]:
score = classification_report(y_test, y_pred)

In [14]:
print(score)

              precision    recall  f1-score   support

           0       0.92      0.93      0.93      3500
           1       0.75      0.71      0.73      1000

    accuracy                           0.88      4500
   macro avg       0.84      0.82      0.83      4500
weighted avg       0.88      0.88      0.88      4500

